In [1]:
# !pip install livelossplot

In [2]:
import pandas as pd
from IPython.display import display
pd.options.display.max_columns = None
pd.options.display.max_rows = None
import missingno as msno
import numpy as np
from datetime import datetime, timedelta
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV, train_test_split, KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score, make_scorer, precision_recall_curve
from sklearn.metrics import confusion_matrix, recall_score, precision_score
from sklearn.multioutput import MultiOutputClassifier
from pathlib import Path
from livelossplot.tf_keras import PlotLossesCallback
import tensorflow as tf
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import accuracy_score
from keras.models import load_model

In [3]:
df = pd.read_csv('train.csv')

In [4]:
df = df[~df['ID'].isin(['54-102-244-02', '54-601-015-02', '54-602-038-01'])]
df = df[~df['ID'].isin(['54-102-340-01', '54-701-024-02', '54-001-079-01'])]
df = df[~df['ID'].isin(['54-102-238-02', '54-701-032-01'])]
df = df[df['ID'] != '54-701-009-01']

In [5]:
test_df = pd.read_csv('test_dataset_test.csv')

In [6]:
df = df.drop(['ID_y'], axis = 1)
df = df.set_index('ID')
test_df = test_df.set_index('ID')

In [7]:
test_df.loc[(test_df['Национальность'] == 'Удмурты') | (test_df['Национальность'] == 'Лезгины'),\
            'Национальность'] = 'Другие национальности'
df.loc[~df['Национальность'].isin(list(test_df['Национальность'].unique())),\
            'Национальность'] = 'Другие национальности'
test_df.loc[(test_df['Религия'] == 'Другое') | (test_df['Религия'] == 'Индуизм'),\
            'Религия'] = 'Нет'
test_df.loc[(test_df['Статус Курения'] == 'Никогда не курил'),\
            'Статус Курения'] = 'Никогда не курил(а)'

In [8]:
df[['Время засыпания', 'Время пробуждения']] = df[['Время засыпания', 'Время пробуждения']].astype('datetime64')
test_df[['Время засыпания', 'Время пробуждения']] = test_df[['Время засыпания', 'Время пробуждения']].astype('datetime64')
df[['Время засыпания', 'Время пробуждения']].dtypes, test_df[['Время засыпания', 'Время пробуждения']].dtypes
df['Время сна'] = df['Время пробуждения'] - df['Время засыпания']
test_df['Время сна'] = test_df['Время пробуждения'] - test_df['Время засыпания']

In [9]:
def sleep_time(x):
    x = int(x.total_seconds()) // 3600
    if x < 0:
        x += 24
    return x
df['Время сна'] = df['Время сна'].apply(lambda x: sleep_time(x))
test_df['Время сна'] = test_df['Время сна'].apply(lambda x: sleep_time(x))

In [10]:
df = df[df['Время сна'] != 19]

In [11]:
df = df.fillna(0)
df[['Возраст курения', 'Сигарет в день', 'Возраст алког']] =\
df[['Возраст курения', 'Сигарет в день', 'Возраст алког']].astype('int64')
test_df = test_df.fillna(0)
test_df[['Возраст курения', 'Сигарет в день', 'Возраст алког']] =\
test_df[['Возраст курения', 'Сигарет в день', 'Возраст алког']].astype('int64')

In [12]:
print(f'В столбце "Пол":  ', end = '')
cat_col = df['Пол'].astype('category')
codes = cat_col.cat.codes
cats = cat_col.cat.categories
df.loc[:, 'Пол'] = codes
print(f'{cats[0]} --> 0,  ', end = '')
print(f'{cats[1]} --> 1.')  

В столбце "Пол":  Ж --> 0,  М --> 1.


In [13]:
print(f'В столбце "Пол":  ', end = '')
cat_col = test_df['Пол'].astype('category')
codes = cat_col.cat.codes
cats = cat_col.cat.categories
test_df.loc[:, 'Пол'] = codes
print(f'{cats[0]} --> 0,  ', end = '')
print(f'{cats[1]} --> 1.')

В столбце "Пол":  Ж --> 0,  М --> 1.


In [14]:
df = pd.get_dummies(df)
test_df = pd.get_dummies(test_df)

In [15]:
df[['Время засыпания', 'Время пробуждения']] = df.loc[:, ['Время засыпания', 'Время пробуждения']].applymap(lambda x: int(x.hour))
test_df[['Время засыпания', 'Время пробуждения']] = test_df.loc[:, ['Время засыпания', 'Время пробуждения']].applymap(lambda x: int(x.hour))

In [16]:
corr_1 = df.corr(method='kendall')[['Артериальная гипертензия', 'ОНМК', 'Стенокардия, ИБС, инфаркт миокарда',\
                                  'Сердечная недостаточность', 'Прочие заболевания сердца']].rename(columns =\
                                    {'Артериальная гипертензия':'АГ1', 'ОНМК':'ОНМК1', 'Стенокардия, ИБС, инфаркт миокарда':\
                                    'СК1, ИБС1, ИМ1', 'Сердечная недостаточность':'СН1', 'Прочие заболевания сердца':\
                                    'ПЗС1'})
corr_2 = df.corr(method='spearman')[['Артериальная гипертензия', 'ОНМК', 'Стенокардия, ИБС, инфаркт миокарда',\
                                  'Сердечная недостаточность', 'Прочие заболевания сердца']].rename(columns =\
                                    {'Артериальная гипертензия':'АГ2', 'ОНМК':'ОНМК2', 'Стенокардия, ИБС, инфаркт миокарда':\
                                    'СК2, ИБС2, ИМ2', 'Сердечная недостаточность':'СН2', 'Прочие заболевания сердца':\
                                    'ПЗС2'})
corr_df = pd.concat([corr_1, corr_2], axis=1)

In [17]:
temp = corr_df.copy()
min_corr = 0.0001
corr_dict = dict()
while len(temp) != 0:
    temp_2 = temp
    temp_2['summ'] = temp_2.applymap(lambda x: 1 if abs(x) <= min_corr else 0).sum(axis = 1)
    temp = temp_2[temp_2['summ'] != 10]
    temp_2 = temp_2[temp_2['summ'] == 10]
    if len(temp_2) != 0:
        corr_dict[tuple(temp_2.index)] = min_corr
    min_corr += 0.0001

In [18]:
df_2 = corr_df
df_2['summ'] = corr_df.applymap(lambda x: 1 if abs(x) <= 0.1 else 0).sum(axis = 1)
df_2_rows = df_2[df_2['summ'] != 10].index
df_2 = df.loc[:, list(df_2_rows)]

In [19]:
hypothesis_list = [df, df_2]

In [20]:
data_list = [MinMaxScaler().fit_transform(hypothesis.drop(columns = ['Артериальная гипертензия', 'ОНМК',\
    'Стенокардия, ИБС, инфаркт миокарда', 'Сердечная недостаточность', 'Прочие заболевания сердца']).values)\
             for hypothesis in hypothesis_list]
targets_list = [MinMaxScaler().fit_transform(hypothesis[['Артериальная гипертензия', 'ОНМК',\
    'Стенокардия, ИБС, инфаркт миокарда', 'Сердечная недостаточность', 'Прочие заболевания сердца']].values)\
             for hypothesis in hypothesis_list]

In [21]:
path = Path("/home/michael/AIchampionship/best_model/")
path.mkdir(exist_ok=True)
file_to_save = "checkpoint_best"  
path_to_save = str(path / file_to_save)

checkpoint = tf.keras.callbacks.ModelCheckpoint(path_to_save,\
            monitor='val_recall', verbose=1, save_best_only=True, step_per_epoch = 100, mode='max')

In [22]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(36, input_dim=73, kernel_initializer='he_uniform', activation='relu'))
model.add(tf.keras.layers.Dense(20, activation='relu'))
model.add(tf.keras.layers.Dense(10, activation='relu'))
model.add(tf.keras.layers.Dense(5, activation='sigmoid'))
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
              loss='binary_crossentropy',
              metrics=[tf.keras.metrics.AUC(), tf.keras.metrics.Recall()])

2022-06-19 21:05:19.569948: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:922] could not open file to read NUMA node: /sys/bus/pci/devices/0000:04:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-06-19 21:05:19.602215: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:922] could not open file to read NUMA node: /sys/bus/pci/devices/0000:04:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-06-19 21:05:19.602590: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:922] could not open file to read NUMA node: /sys/bus/pci/devices/0000:04:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-06-19 21:05:19.603405: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 36)                2664      
                                                                 
 dense_1 (Dense)             (None, 20)                740       
                                                                 
 dense_2 (Dense)             (None, 10)                210       
                                                                 
 dense_3 (Dense)             (None, 5)                 55        
                                                                 
Total params: 3,669
Trainable params: 3,669
Non-trainable params: 0
_________________________________________________________________


In [24]:
df_NN = df.copy()

In [ ]:
df_NN_1 = df_NN[(df_NN['Артериальная гипертензия'] == 0) & (df_NN['Стенокардия, ИБС, инфаркт миокарда'] == 0)]
df_NN_1.iloc[:, 23:28].to_numpy().sum(axis=0)

In [ ]:
df_NN_2 = df_NN[(df_NN['Артериальная гипертензия'] == 0) & (df_NN['Стенокардия, ИБС, инфаркт миокарда'] == 0) & (df_NN['Прочие заболевания сердца'] == 0)]
df_NN_2.iloc[:, 23:28].to_numpy().sum(axis=0)

In [ ]:
df_NN_3 = df_NN[(df_NN['Артериальная гипертензия'] == 0) & (df_NN['Прочие заболевания сердца'] == 0)]
df_NN_3.iloc[:, 23:28].to_numpy().sum(axis=0)

In [ ]:
for i in range(40):
    if i < 10:
        df_NN = pd.concat([df_NN, df_NN_1])
    elif i < 30:
        df_NN = pd.concat([df_NN, df_NN_3])
    else:
        df_NN = pd.concat([df_NN, df_NN_2])

In [ ]:
df_NN.iloc[:, 23:28].to_numpy().sum(axis=0)

In [ ]:
df_NN = df_NN.sample(frac = 1)

In [ ]:
df_NN.shape

In [25]:
X_NN = MinMaxScaler().fit_transform(df_NN.drop(columns = ['Артериальная гипертензия', 'ОНМК',\
    'Стенокардия, ИБС, инфаркт миокарда', 'Сердечная недостаточность', 'Прочие заболевания сердца']).values)
y_NN = MinMaxScaler().fit_transform(df_NN[['Артериальная гипертензия', 'ОНМК',\
    'Стенокардия, ИБС, инфаркт миокарда', 'Сердечная недостаточность', 'Прочие заболевания сердца']].values)

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X_NN, y_NN, 
                                                    train_size=0.75, 
                                                    random_state=17)

In [ ]:
EPOCHS = 100
history = model.fit(
    X_train, y_train,
    epochs=EPOCHS,
    validation_data=(X_test, y_test),
    steps_per_epoch=200,
    callbacks=[PlotLossesCallback(), checkpoint])

In [ ]:
accuracy_score(y_test, model.predict(X_test).round())

In [ ]:
roc_auc_score(y_test, model.predict(X_test).round())

In [ ]:
recall_score(y_test, model.predict(X_test).round(), average = 'macro')

In [ ]:
type(model.predict(X_test).round())

In [ ]:
columns = ['Артериальная гипертензия', 'ОНМК',\
    'Стенокардия, ИБС, инфаркт миокарда', 'Сердечная недостаточность', 'Прочие заболевания сердца']
target_df = pd.DataFrame(model.predict(X_test_NN).round(), columns = columns, index = test_df.index)

In [ ]:
X_test_NN = MinMaxScaler().fit_transform(test_df.values)

In [ ]:
test_df

In [ ]:
target_df.reset_index(inplace = True)

In [ ]:
target_df.iloc[:, 1:] = target_df.iloc[:, 1:].astype('int64')

In [ ]:
target_df

In [ ]:
target_df.to_csv('fourth_proba.csv', index=False)

In [ ]:
model = load_model('/home/michael/AIchampionship/best_model/checkpoint_best/')

In [ ]:
model_1.predict(X_test_NN).round()